In [47]:
# | default_exp backend.llm_factory

In [53]:
# | export
import os
import re
from openai import OpenAI
from rich.console import Console
from rich.markdown import Markdown
from agentic.configs.loader import get_model_config, get_settings_config, get_reasoning_config

In [49]:
# | export
RESET = "\033[0m"
# Get reasoning config
reasoning_config = get_reasoning_config()
show_thinking = reasoning_config.get('show_thinking', True)
thinking_color = reasoning_config.get('thinking_color', 'pink')

# Set color based on config with enhanced colors
color_codes = {
            'pink': "\033[38;2;200;100;120m",     # Darker pink
            'blue': "\033[38;2;70;130;180m",      # Steel blue
            'green': "\033[38;2;60;179;113m",     # Medium sea green
            'yellow': "\033[38;2;204;204;0m",     # Olive-like dark yellow
            'purple': "\033[38;2;147;112;219m",   # Medium purple
            'cyan': "\033[38;2;0;139;139m"        # Dark cyan
        }

color_code = color_codes.get(thinking_color.lower(), color_codes['green'])

In [50]:
# | export
def render_math_in_thinking(text: str) -> str:
    """Render mathematical formulas in a terminal-friendly format using Unicode."""
    
    # Greek letters and math symbols
    math_replacements = {
        r'\bsigma\b': 'σ',
        r'\bmu\b': 'μ',
        r'\bpi\b': 'π',
        r'\bsum\b': '∑',
        r'\bsqrt\b': '√',
        r'\balpha\b': 'α',
        r'\bbeta\b': 'β',
        r'\bgamma\b': 'γ',
        r'\bdelta\b': 'δ',
        r'\btheta\b': 'θ',
        r'\blambda\b': 'λ',
        r'\bphi\b': 'φ',
        r'\bapprox\b': '≈',
        r'\bnot equal\b': '≠',
        r'\bless than or equal to\b': '≤',
        r'\bgreater than or equal to\b': '≥',
        r'\binfty\b': '∞',
    }

    # Apply replacements
    for pattern, replacement in math_replacements.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

    # Superscripts
    superscripts = {
        '0': '⁰', '1': '¹', '2': '²', '3': '³',
        '4': '⁴', '5': '⁵', '6': '⁶',
        '7': '⁷', '8': '⁸', '9': '⁹',
        '+': '⁺', '-': '⁻', '=': '⁼', '(': '⁽', ')': '⁾',
    }

    def superscriptify(match):
        return ''.join(superscripts.get(ch, ch) for ch in match.group(1))

    text = re.sub(r'\*\*([0-9\+\-\=\(\)]+)', superscriptify, text)

    # Fractions (simple 1/2 style)
    text = re.sub(r'\b1/2\b', '½', text)
    text = re.sub(r'\b1/4\b', '¼', text)
    text = re.sub(r'\b3/4\b', '¾', text)

    return text

    
def show_thinking_header():
    """Display beautiful thinking header"""
    if show_thinking:
        print(f"\n{color_code}╭─────────────────────── 🤔 Thinking ───────────────────────╮{RESET}")

def show_thinking_footer():
    """Display beautiful thinking footer"""
    if show_thinking:
        print(f"{color_code}╰────────────────────────────────────────────────────────────╯{RESET}")

In [51]:
# | export
class LLMClient:
    def __init__(self, model=None, base_url=None, api_key=None):
        # Load config defaults
        model_config = get_model_config()
        settings_config = get_settings_config()
        
        # Use config values as defaults
        if model is None:
            model = model_config.get('name', 'qwen3:8b')
        if base_url is None:
            base_url = model_config.get('url') or os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434/v1')
        if api_key is None:
            api_key = model_config.get('api_key', 'ollama')
        
        # Store config settings
        self.temperature = settings_config.get('temperature', 0.7)
        self.max_tokens = settings_config.get('max_tokens', 10000)
        timeout = model_config.get('timeout', 300.0)
        
        # Support both OpenAI and Ollama
        if base_url == "openai":
            # Use OpenAI directly
            self.client = OpenAI(api_key=api_key)
            self.model = model if model != "gpt-oss:20b" else "gpt-4"
        else:
            # Use Ollama or other OpenAI-compatible endpoint
            self.client = OpenAI(
                base_url=base_url,
                api_key=api_key, 
                timeout=timeout
            )
            self.model = model
    
    def create_completion(self, messages, tools=None, stream=True):
        """Create chat completion with OpenAI API"""
        return self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            stream=stream,
            temperature=self.temperature
        )

    
    def process_response(self, response, console):
        """Process non-streaming response with tool calls"""
        message = response.choices[0].message

        # Don't save thinking in the history
        
        cleaned_content = ""
        if hasattr(message, 'reasoning') and message.reasoning:
            show_thinking_header()
            console.print(Markdown(message.reasoning))
            show_thinking_footer()
        if hasattr(message, 'content') and message.content:
            content = message.content
    
            # Extract <think>...</think> content
            think_matches = re.findall(r"<think>(.*?)</think>", content, re.DOTALL)
    
            # Remove <think>...</think> blocks from original content
            cleaned_content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)

            # Print each <think> block inside the fancy box
            for think in think_matches:
                think = think.strip()
                if think:
                    show_thinking_header()
                    print(f"{color_code}{think}{RESET}")
                    show_thinking_footer()
    
            # Print main message content
            if cleaned_content.strip():
                console.print(Markdown(cleaned_content.strip()))
 
        if hasattr(message, 'tool_calls') and message.tool_calls:
            tool_calls = []
            for tool_call in message.tool_calls:
                tool_calls.append({
                    "id": tool_call.id,
                    "type": tool_call.type,
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    }
                })
            
            executed_calls = self._execute_tool_calls(tool_calls)
            return {"content": cleaned_content.strip(), "tool_calls": executed_calls}
        
        return {"content": cleaned_content.strip(), "tool_calls": []}

    
    def handle_streaming_response(self, response, console: Console):
        """Handle streaming response with streaming Markdown and think-tag rendering."""
        
        full_content = ""
        tool_calls = []
        first_token = True
        in_think_tag = False
        think_buffer = ""
        markdown_buffer = ""
        markdown_line_buffer = ""
        think_started = False
        
        def show_thinking_content(content):
            nonlocal think_started
            if show_thinking:
                if not think_started:
                    show_thinking_header()
                    print(f"{color_code}│ ", end="", flush=True)
                    think_started = True
                
                # enhanced_content = render_math_in_thinking(content)
                print(f"{color_code}{content}{RESET}", end="", flush=True)
            return think_started
    
        def flush_markdown_line():
            """Flush buffered markdown line if it ends with a newline."""
            nonlocal markdown_line_buffer
            if markdown_line_buffer.endswith('\n'):
                try:
                    console.print(Markdown(markdown_line_buffer.strip()))
                except:
                    print(markdown_line_buffer.strip())
                markdown_line_buffer = ""
    
        for chunk in response:
            if chunk.choices and chunk.choices[0].delta:
                delta = chunk.choices[0].delta
    
                # Reasoning content (not added to final message)
                if hasattr(delta, 'reasoning') and delta.reasoning:
                    think_started = show_thinking_content(delta.reasoning)
    
                if hasattr(delta, 'content') and delta.content:
                    content = delta.content
    
                    if '<think>' in content:
                        in_think_tag = True
                        parts = content.split('<think>')
                        if parts[0]:
                            markdown_line_buffer += parts[0]
                            full_content += parts[0]
                        flush_markdown_line()
                        think_buffer = parts[1] if len(parts) > 1 else ""
                        if think_buffer:
                            think_started = show_thinking_content(think_buffer)
                        continue
    
                    if '</think>' in content:
                        in_think_tag = False
                        parts = content.split('</think>')
                        if parts[0]:
                            think_buffer += parts[0]
                            think_started = show_thinking_content(parts[0])
                        if think_started:
                            print(f"{color_code} │{RESET}")
                            show_thinking_footer()
                            think_started = False
                        think_buffer = ""
                        if len(parts) > 1:
                            markdown_line_buffer += parts[1]
                            full_content += parts[1]
                            flush_markdown_line()
                        continue
    
                    if in_think_tag:
                        think_buffer += content
                        think_started = show_thinking_content(content)
                    else:
                        markdown_line_buffer += content
                        full_content += content
                        flush_markdown_line()
    
                    if first_token:
                        first_token = False
    
                # Tool calls
                if hasattr(delta, 'tool_calls') and delta.tool_calls:
                    for tool_call_delta in delta.tool_calls:
                        if tool_call_delta.index is not None:
                            while len(tool_calls) <= tool_call_delta.index:
                                tool_calls.append({
                                    "id": "",
                                    "type": "function",
                                    "function": {"name": "", "arguments": ""}
                                })
    
                            current_tool_call = tool_calls[tool_call_delta.index]
                            if tool_call_delta.id:
                                current_tool_call["id"] = tool_call_delta.id
                            if tool_call_delta.function:
                                if tool_call_delta.function.name:
                                    current_tool_call["function"]["name"] = tool_call_delta.function.name
                                if tool_call_delta.function.arguments:
                                    current_tool_call["function"]["arguments"] += tool_call_delta.function.arguments
    
        # Flush any remaining markdown
        if markdown_line_buffer.strip():
            try:
                console.print(Markdown(markdown_line_buffer.strip()))
            except:
                print(markdown_line_buffer.strip())
    
        return {"content": full_content, "tool_calls": tool_calls}


In [52]:
# Example usage 

system_prompt = {
    "role": "system",
    "content": (
        "You are a helpful assistant. "
        "Return all your responses using valid **Markdown syntax**, including:\n"
        "- Headers (`#`, `##`)\n"
        "- Bullet points\n"
        "- Code blocks (triple backticks)\n"
        "- Bold / italic text\n"
        "- Quotes and tables if needed\n\n"
        "Also, try to use formatting that works well with color rendering in terminals using `rich`."
    )
}



from rich.console import Console
client = LLMClient()
response = client.create_completion(
    messages=[
    system_prompt,
    {"role": "user", "content": "Explain how a neural network works."}
    ],
    tools=[],
    stream=True
)
result = client.handle_streaming_response(response,Console() )
# result = client.process_response(response,Console())


╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, I need to explain how a neural network works. Let me start by recalling what I know. Neural networks are inspired by the human brain. They consist of layers of interconnected nodes, or neurons. Each neuron receives input, processes it, and passes the output to the next layer. The basic components are input layer, hidden layers, and output layer. Each connection between neurons has a weight, which determines the strength of the signal. There's also a bias term that helps shift the activation function.

Wait, maybe I should structure the explanation step by step. First, describe the structure: input, hidden, output layers. Then explain neurons and how they process inputs. Then the role of weights and biases. Then activation functions, which introduce non-linearity. Then the process of forward propagation, where data moves through the network. Then training with backpropagation and optimization. Maybe mention loss func

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                           How a Neural Network Works                                            ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Neural networks are computational models inspired by the structure and function of the human brain. They are       
designed to recognize patterns, make predictions, and solve complex problems by learning from data. Below is a     
step-by-step explanation of how they work:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

1. Structure of a Neural Network

...

Neural networks learn by:

 1 Applying transformations to input data through layers of neurons.

 2 Using activation functions to introduce non-linearity.

 3 Minimizing a loss function through backpropagation and optimization.

They are powerful tools for tasks like image recognition, natural language processing, and predictive analytics.